In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data                                        # mnist data set
import numpy as np

In [2]:
import math

"""mini-batch"""
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    output_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        output_batches.append(batch)
        
    return output_batches

In [3]:
"""hyper-parameters define"""
batch_size = 128                                            # mini-batch size
epochs = 100                                                # epoch size
learn_rate = 0.1


"""data importation"""
mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)
# The features are already scaled and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# mini-batch operation
train_batches = batches(batch_size, train_features, train_labels)

"""model parameters define"""
n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)


"""TensorFlow frame parameters define"""
# Features and Labels pre-define
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Logits is composed of weights and bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))           # weights
bias = tf.Variable(tf.random_normal([n_classes]))                       # bias
logits = tf.add(tf.matmul(features, weights), bias)                     # logis

# Learning  rate pre-define
learning_rate = tf.placeholder(tf.float32)

# Define loss by logits and labels
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
# Define optimizer by learning rate and loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy by 'equal', 'cast' and 'reduce_mean' functions
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()                    # globle variables pre-define

"""TensorFlow Session(main processing)"""
with tf.Session() as sess:
    sess.run(init)                                         # globle variables define

    # Training ALL data 'epoch' times
    for epoch_i in range(epochs):

        # Training data BATCH-by-BATCH a time
        for batch_features, batch_labels in train_batches:
            # re-define 'feed_dict' each time(used to transfer every batch of data included features and labels)
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            # run optimizer by 'feed_dict' to optimize model
            sess.run(optimizer, feed_dict=train_feed_dict)
            
        # Calculate loss by every batch of labels
        current_cost = sess.run(cost, feed_dict={features: batch_features, labels: batch_labels})

        # Calculate accuracy by validation data
        valid_accuracy = sess.run(accuracy, feed_dict={features: valid_features, labels: valid_labels})

        # epoch_i, sess, batch_features, batch_labels
        
        if epoch_i%10==0:
            print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
            epoch_i,
            current_cost,
            valid_accuracy))
        
    # Calculate accuracy for test dataset(target accuracy)
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})
print('Test Accuracy: {}'.format(test_accuracy))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /datasets/ud730/mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /datasets/ud730/mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /datasets/ud730/mnist\t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_